In [ ]:
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from urllib.parse import quote

In [ ]:
DIR_DATA = Path.cwd().parents[3] / "Database/data/mse-daily-data"

In [ ]:
load_dotenv()  

PGUSER = os.getenv("PGUSER")
PGPASSWORD = os.getenv("PGPASSWORD")
PGHOST = os.getenv("PGHOST", "localhost")
PGPORT = os.getenv("PGPORT", "5432")
# PGDATABASE = os.getenv("PGDATABASE")
PGDATABASEMSE = os.getenv("PGDATABASEMSE")

PGPASSWORD_ENC = quote(PGPASSWORD)

In [ ]:


connection_string = f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD_ENC}@{PGHOST}:{PGPORT}/{PGDATABASEMSE}"
print("Using connection string (masked):", f"postgresql+psycopg2://{PGUSER}:***@{PGHOST}:{PGPORT}/{PGDATABASEMSE}")

engine = create_engine(connection_string, pool_pre_ping=True)

with engine.connect() as conn:
    who = conn.execute(text("SELECT current_user, current_database();")).fetchone()
    print("Connected as:", who)


In [ ]:

all_files = [f for f in os.listdir(DIR_DATA) if f.endswith('.csv')]

df_list = []
for file in all_files:
    file_path = os.path.join(DIR_DATA, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# Combine all files into one DataFrame
mse_combined_data = pd.concat(df_list, ignore_index=True)
mse_combined_data['counter_id'] = range(1, len(mse_combined_data) + 1)
mse_combined_data.head()


In [ ]:
mse_combined_data.head(20)

In [ ]:
company_map = {
    "AIRTEL": "AIRTEL MALAWI PLC",
    "BHL": "BLANTYRE HOTELS PLC",
    "FDHB": "FDH BANK PLC",
    "FMBCH": "FMB CAPITAL HOLDINGS PLC",
    "ICON": "ICON PROPERTIES PLC",
    "ILLOVO": "ILLOVO SUGAR MALAWI PLC",
    "MPICO": "MPICO PLC",
    "NBM": "NATIONAL BANK OF MALAWI",
    "NBS": "NBS BANK PLC",
    "NICO": "NICO HOLDINGS PLC",
    "NITL": "NATIONAL INVESTMENT TRUST PLC",
    "OMU": "OLD MUTUAL LIMITED",
    "PCL": "PRESS CORPORATION PLC",
    "STANDARD": "STANDARD BANK MALAWI PLC",
    "SUNBIRD": "SUNBIRD TOURISM PLC",
    "TNM": "TELEKOM NETWORKS MALAWI PLC"
}

# Add a new column with the full name
mse_combined_data["name"] = mse_combined_data["counter"].map(company_map)

In [ ]:
mse_combined_data = mse_combined_data.rename(columns={
    "counter": "ticker",
        "trade_date": "date_listed",
        "buy_price": "listing_price"
})

selected_columns = ["counter_id", "ticker", "name", "date_listed", "listing_price"]
counters_df = mse_combined_data[selected_columns]
counters_df.to_csv(DIR_DATA.parent / "mse_selected.csv", index=False)

In [ ]:
price_daily_columns = {
    "counter_id": "counter_id",
    "date_listed": "trade_date",
    "listing_price": "open_mwk",
    "daily_range_high": "high_mwk",
    "daily_range_low": "low_mwk",
    "today_closing_price": "close_mwk",
    "volume_traded": "volume"
}

prices_daily_df = mse_combined_data.rename(columns=price_daily_columns)[list(price_daily_columns.values())]
prices_daily_df.to_csv(DIR_DATA.parent / "price_daily_data.csv", index=False)

In [ ]:
# 1️⃣ Fetch existing counter_ids from the database
with engine.connect() as conn:
    existing_ids = conn.execute(text("SELECT counter_id FROM counters")).fetchall()
existing_ids = {row[0] for row in existing_ids}

# 2️⃣ Keep only new rows that are not already in DB
counters_df_new = counters_df[~counters_df['counter_id'].isin(existing_ids)]

# 3️⃣ Insert new rows in chunks
chunksize = 1000
for start in range(0, len(counters_df_new), chunksize):
    counters_df_new.iloc[start:start+chunksize].to_sql(
        "counters",
        con=engine,
        if_exists="append",
        index=False,
        method="multi"
    )

print(f"{len(counters_df_new)} new rows added to the database.")



In [ ]:
chunksize = 1000
for start in range(0, len(counters_df), chunksize):
    counters_df.iloc[start:start+chunksize].to_sql(
        "counters", con=engine, if_exists="append", index=False, method="multi"
    )


In [ ]:
chunksize = 1000
for start in range(0, len(prices_daily_df), chunksize):
    prices_daily_df.iloc[start:start+chunksize].to_sql(
        "prices_daily", con=engine, if_exists="append", index=False, method="multi"
    )

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM counters;")).fetchone()
    print("Total rows in counters:", result[0])

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM prices_daily;")).fetchone()
    print("Total rows in prices_daily:", result[0])

In [ ]:
# read the entire 'counters' table
df_counters = pd.read_sql("SELECT * FROM counters LIMIT 5;", con=engine)

df_mse = pd.read_sql("SELECT * FROM prices_daily LIMIT 5;", con=engine)

# Print the DataFrame
print(df_counters)
print("*************************")
print(df_mse)

In [ ]:
# SQL query
query = """
SELECT ticker, name, date_listed
FROM counters
WHERE date_listed IS NOT NULL
ORDER BY date_listed
LIMIT 20;
"""

# Execute the query and load results into a pandas DataFrame
df_listed = pd.read_sql(query, con=engine)

# Print the results nicely
print("Counters listed on the Malawi Stock Exchange:")
print(df_listed)

In [ ]:
# SQL query 
query = """
SELECT ticker, name, MIN(date_listed) AS first_listed
FROM counters
WHERE date_listed IS NOT NULL
GROUP BY ticker, name
ORDER BY first_listed
LIMIT 20;
"""

df_first_listed = pd.read_sql(query, con=engine)

print("First occurrence of each counter on the Malawi Stock Exchange:")
print(df_first_listed)

In [ ]:
query = """
SELECT c.ticker, c.name, p.trade_date, p.close_mwk AS closing_price
FROM counters c
JOIN prices_daily p ON c.counter_id = p.counter_id
WHERE p.trade_date = '2025-09-09'
ORDER BY c.ticker;
"""

# Execute query and load into pandas
df_closing = pd.read_sql(query, con=engine)

# Print results
print("Closing prices on 2025-09-09:")
print(df_closing)